## process PDF and CSV file and build an Azure AI Search index as knowledge base

### 0. Preparation

#### 0.1 import the modules

In [10]:
from openai import AzureOpenAI
import os
import json
import csv
import io
from dotenv import load_dotenv
load_dotenv()
from PyPDF2 import PdfReader, PdfWriter
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import ContentFormat
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.search.documents.models import QueryType,VectorizedQuery
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticPrioritizedFields,
    SemanticField,  
    SearchField,  
    SemanticSearch,
    VectorSearch,  
    HnswAlgorithmConfiguration,
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SemanticField,  
    SearchField,  
    VectorSearch,  
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)

#### 0.2 Configurations setting

In [2]:
# Define the local folders
pdf_folder = "./documents/PDFs" # put your PDFs in this folder
output_folder = "./documents/jsons"
csv_folder = './documents/CSVs'  # put your CSVs in this folder
JSON_FOLDER = output_folder
JSON_UPLOAD_ERROR_FOLDER = "./documents/json_upload_error"

# Configure Azure AI Search setting 
azure_ai_search_service_endpoint = os.getenv("AZURE_AI_SEARCH_SERVICE_ENDPOINT") 
azure_ai_search_index_name = os.getenv("AZURE_AI_SEARCH_INDEX_NAME") 
azure_ai_search_admin_key = os.getenv("AZURE_AI_SEARCH_ADMIN_KEY") 
azure_ai_search_semantic_configuration_name = os.getenv("AZURE_AI_SEARCH_SEMANTIC_CONFIG_NAME")

# Configure Azure AI Document Intelligence setting
Azure_DI_Endpoint = os.getenv("AZURE_DI_ENDPOINT")
Azure_DI_Key = os.getenv("AZURE_DI_KEY")
endpoint = Azure_DI_Endpoint
key = Azure_DI_Key
document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

# Configuration Azure OpenAI setting
api_base = os.getenv("AOAI_API_BASE") # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
api_key = os.getenv("AOAI_API_KEY")
embedding_deployment_name = os.getenv("EMBEDDING_MODEL_NAME")
api_version = "2024-02-01" # this might change in the future 2023-12-01-preview 2024-02-15-preview
# instantiate the AzureOpenAI client
client = AzureOpenAI(
    api_key=api_key,  
    api_version=api_version,
    azure_endpoint = api_base,
)

#### 0.3 Helper Functions: create embedding for a text string input

In [3]:
# Function to generate embeddings for the question field
def create_embedding(context: str):
  model = embedding_deployment_name
  response = client.embeddings.create(input=context, model=model).data[0].embedding
  return response

### 1. Process source files

#### 1.1 process the PDF file (Convert a folder of PDF files into JSON files per page)

In [6]:
def convert_onepager_pdf_io_to_markdown (pdf_io):
    Azure_DI_Endpoint = os.getenv("AZURE_DI_ENDPOINT")
    Azure_DI_Key = os.getenv("AZURE_DI_KEY")
    endpoint = Azure_DI_Endpoint
    key = Azure_DI_Key
    document_intelligence_client = DocumentIntelligenceClient(endpoint, credential = AzureKeyCredential(key))
    # Convert the in-memory PDF page to Markdown
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout", 
        analyze_request=pdf_io, 
        output_content_format=ContentFormat.MARKDOWN, 
        content_type="application/pdf"
    )
    result = poller.result()
    markdown_content = result.content
    return markdown_content

def chunk_pdf_to_paginated_JSONs(pdf_path, output_folder, start_page=1):
    # Open the PDF file
    with open(pdf_path, "rb") as file:
        pdf = PdfReader(file)
        
        # Adjust start_page to be zero-based index for page enumeration
        start_page_index = start_page - 1

        # Process each page starting from page n
        for page_number, page in enumerate(pdf.pages):
            if page_number < start_page_index:
                continue
            # Create a new PDF writer and add the current page
            output_pdf = PdfWriter()
            output_pdf.add_page(page)

            # Save the chunked PDF to a BytesIO object (in memory)
            output_pdf_io = io.BytesIO()
            output_pdf.write(output_pdf_io)
            output_pdf_io.seek(0) # Reset the pointer to the beginning of the BytesIO object

            # Convert the in-memory PDF page to Markdown
            markdown_content = convert_onepager_pdf_io_to_markdown(output_pdf_io)
            embedding = create_embedding(markdown_content)
            source = f"{os.path.splitext(os.path.basename(pdf_path))[0]}_page{page_number + 1}"
            category = "ACS880" # change this for other machine categories


            # Create the JSON filename based on the input path
            output_filename = f"{os.path.splitext(os.path.basename(pdf_path))[0]}_page{page_number + 1}.json"
            output_path = os.path.join(output_folder, output_filename)
            # Write to a JSON file with keys "content", "embedding", "source", "category"
            with open(output_path, mode='w', encoding='utf-8') as json_file:
                print(f"Writing JSON file for page {page_number + 1}")
                json.dump({"category": category, "source": source, "content": markdown_content, "embedding": embedding}, json_file, ensure_ascii=False, indent=4)

            # release the memory
            output_pdf_io.close()

def process_pdf_folder(pdf_folder, output_folder, start_page=1):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    # Get a list of all PDF files in the input folder
    pdf_files = [file for file in os.listdir(pdf_folder) if file.endswith(".pdf")]
    # Process each PDF file
    for pdf_file in pdf_files:
        # Open the PDF file
        print(f"Processing {pdf_file}")
        input_path = os.path.join(pdf_folder, pdf_file)
        chunk_pdf_to_paginated_JSONs(input_path, output_folder, start_page)
    return print("PDFs chunked to paginated JSONs successfully")

process_pdf_folder(pdf_folder, output_folder)

Processing POC Proposal - Ask What Matters.pdf
Writing JSON file for page 1
Writing JSON file for page 2
Writing JSON file for page 3
PDFs chunked to paginated JSONs successfully


#### 1.2 process CSV file (convert CSV file into JSON files)


In [12]:
# Dictionary to map Chinese headers to English headers
header_translation = {
    "﻿分类": "category",
    "问题": "question",
    "相似问题": "similar_question",
    "标准答案": "standard_answer",
    "附件": "attachment",
    "TPM-Revise": "TPM-Revise",
}
# Function to process the answer field
def trim_answer(answer):
    # Remove new line characters and spaces
    answer = answer.replace('\n', ' ').replace('\r', ' ').replace(" ", "")
    # extract the answer from the string delimited by "A:" or "答案: or A： or 答案："
    if "A:" in answer:
        return answer.split("A:", 1)[1].strip()
    elif "A：" in answer:
        return answer.split("A：", 1)[1].strip()
    elif "答案:" in answer:
        return answer.split("答案:", 1)[1].strip()
    elif "答案：" in answer:
        return answer.split("答案：", 1)[1].strip()
    else:
        return answer

# Function to trim the category field
def trim_category(category):
    # Remove new line characters and spaces
    category = category.split(' ', 1)[0]
    return category
    
# Read the CSV file as a list within the csv folder
csv_files = [file for file in os.listdir(csv_folder) if file.endswith(".csv")]
# Process each csv file
for csv_file in csv_files:
    # Open the csv file
    print(f"Processing {csv_file}")
    csv_file_path = os.path.join(csv_folder, csv_file)
    with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for i, row in enumerate(csv_reader):
            # Create a new dictionary with translated headers
            translated_row = {header_translation[key]: value for key, value in row.items()}
            # Process the answer field
            print(f"processing file {i+1}" )
            translated_row['category'] = trim_category(translated_row['category'])
            translated_row['standard_answer'] = trim_answer(translated_row['standard_answer'])
            if translated_row['similar_question'] == "":
                translated_row['similar_question'] = translated_row["question"]
            translated_row['source'] = f"FAQ_Q{i+1}"
            # Generate embeddings for the question field
            if translated_row['question'] != translated_row['similar_question']:
                translated_row['content'] = f"问题：{translated_row['question']} / {translated_row['similar_question']} 答案：{translated_row['standard_answer']}"
                translated_row['embedding'] = create_embedding(f"{translated_row['question']} / {translated_row['similar_question']}")
            else:
                translated_row['content'] = f"问题：{translated_row['question']} 答案：{translated_row['standard_answer']}"
                translated_row['embedding'] = create_embedding(translated_row['question'])
            # remove the field attachment, TPM-Revise, questions, similar_question, standard_answer
            del translated_row['attachment']
            del translated_row['TPM-Revise']
            del translated_row['question']
            del translated_row['similar_question']
            del translated_row['standard_answer']

            # Create the JSON filename based on the row index
            json_file_path = os.path.join(output_folder, f'FAQ_Q{i+1}.json')
            
            # Write the row data to a JSON file
            with open(json_file_path, mode='w', encoding='utf-8') as json_file:
                json.dump(translated_row, json_file, ensure_ascii=False, indent=4)

print("JSON files have been created successfully.")


Processing 知识库 ACS880FAQ.csv
processing file 1
processing file 2
processing file 3
processing file 4
processing file 5
processing file 6
processing file 7
processing file 8
processing file 9
processing file 10
processing file 11
processing file 12
processing file 13
processing file 14
processing file 15
processing file 16
processing file 17
processing file 18
processing file 19
processing file 20
processing file 21
processing file 22
processing file 23
processing file 24
processing file 25
processing file 26
processing file 27
processing file 28
processing file 29
processing file 30
processing file 31
processing file 32
processing file 33
processing file 34
processing file 35
processing file 36
processing file 37
processing file 38
processing file 39
processing file 40
processing file 41
processing file 42
processing file 43
processing file 44
processing file 45
processing file 46
processing file 47
processing file 48
processing file 49
processing file 50
JSON files have been created s

### 2. Index Preparation

#### 2.1 Create AI Search Index using JSON files

##### Prerequisite: there is already a Azure AI Search service resource created in Azure Subscription, so that the AI Search endpoint and Admin-Key are already available 

In [13]:
# Create the search index client
def create_search_index():
    credential = AzureKeyCredential(azure_ai_search_admin_key)
    index_client = SearchIndexClient(
        endpoint=azure_ai_search_service_endpoint, credential=credential)
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True,retrievable=False),
        SearchableField(name="content", type=SearchFieldDataType.String, retrievable=True),
        SearchableField(name="category", type=SearchFieldDataType.String, retrievable=True, filterable=True),
        SearchableField(name="source", type=SearchFieldDataType.String, retrievable=True, filterable=True),
        SearchField(name="embedding", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    searchable=True, vector_search_dimensions=1536, retrievable=False,
                    vector_search_profile_name="vector-config-profile")
    ]
    # Configure the vector search configuration  
    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="myHnsw",
                kind=VectorSearchAlgorithmKind.HNSW,
                parameters=HnswParameters(
                    m=4,
                    ef_construction=400,
                    ef_search=500,
                    metric=VectorSearchAlgorithmMetric.COSINE
                )
            ),
        ],
        profiles=[
            VectorSearchProfile(
                name="vector-config-profile",
                algorithm_configuration_name="myHnsw",
            ),
        ]
    )
    # Configure the semantic search configuration for BM25-reranker
    semantic_config = SemanticConfiguration(
        name=azure_ai_search_semantic_configuration_name,
        prioritized_fields=SemanticPrioritizedFields(
            title_field=SemanticField(field_name="source"),
            content_fields=[SemanticField(field_name="content")],
            keywords_fields=[SemanticField(field_name="category")]
        )
    )
    semantic_search = SemanticSearch(configurations=[semantic_config])

    # Create the search index with the semantic settings
    index = SearchIndex(name=azure_ai_search_index_name, fields=fields,
                        vector_search=vector_search, 
                        semantic_search=semantic_search)
    result = index_client.create_or_update_index(index)
    return(print(f' {result.name} created'))

create_search_index()


retrievable is not a known attribute of class <class 'azure.search.documents.indexes.models._index.SearchField'> and will be ignored


 abbmotiontest created


#### 2.2. load JSON files into index

In [22]:
# Iterate through JSON files and upload documents
def upload_documents():
    credential = AzureKeyCredential(azure_ai_search_admin_key)
    search_client = SearchClient(endpoint=azure_ai_search_service_endpoint, index_name=azure_ai_search_index_name, credential=credential)
    # Create the error folder if it doesn't exist
    if not os.path.exists(JSON_UPLOAD_ERROR_FOLDER):
        os.makedirs(JSON_UPLOAD_ERROR_FOLDER)
    i = 0
    # Iterate over files in the json folder
    for filename in os.listdir(JSON_FOLDER):
        # Construct the json path
        json_file_path = os.path.join(JSON_FOLDER, filename)
        # try uploading the document, if it fails, print the error message and move the document to the error folder and continue
        try:
            with open(json_file_path, "r", encoding="utf-8") as file:
                document = json.load(file)
                # add the id field with the iteration number converted to string
                document["id"] = str(i+1)
                search_client.upload_documents(documents=[document])
                print(f"Uploaded {filename}")
                i += 1
        except Exception as e:
            print(f"Error uploading {filename}: {e}")
            # move the document to the error folder
            try:
                os.rename(json_file_path, os.path.join(JSON_UPLOAD_ERROR_FOLDER, filename))
            except Exception as e:
                print(f"Error moving {filename} to error folder: {e}")
            continue
    return print(f"Uploaded {i} documents")

upload_documents()

Uploaded CN_ACS880 FW manual_revD_page117.json
Uploaded CN_ACS880 FW manual_revD_page118.json
Uploaded CN_ACS880 FW manual_revD_page119.json
Uploaded CN_ACS880 FW manual_revD_page12.json
Uploaded CN_ACS880 FW manual_revD_page120.json
Uploaded CN_ACS880 FW manual_revD_page121.json
Uploaded CN_ACS880 FW manual_revD_page122.json
Uploaded CN_ACS880 FW manual_revD_page123.json
Uploaded CN_ACS880 FW manual_revD_page124.json
Uploaded CN_ACS880 FW manual_revD_page125.json
Uploaded CN_ACS880 FW manual_revD_page126.json
Uploaded CN_ACS880 FW manual_revD_page127.json
Uploaded CN_ACS880 FW manual_revD_page128.json
Uploaded CN_ACS880 FW manual_revD_page129.json
Uploaded CN_ACS880 FW manual_revD_page13.json
Uploaded CN_ACS880 FW manual_revD_page130.json
Uploaded CN_ACS880 FW manual_revD_page131.json
Uploaded CN_ACS880 FW manual_revD_page132.json
Uploaded CN_ACS880 FW manual_revD_page133.json
Uploaded CN_ACS880 FW manual_revD_page134.json
Uploaded CN_ACS880 FW manual_revD_page135.json
Uploaded CN_ACS

#### 2.3 Test the search index

In [14]:
# test the created index with a hybrid search query using semantic reranker
def search_ai_index(question:str):
    credential = AzureKeyCredential(azure_ai_search_admin_key)
    search_client = SearchClient(endpoint=azure_ai_search_service_endpoint, index_name=azure_ai_search_index_name, credential=credential)
    vector_query = VectorizedQuery(vector=create_embedding(question), k_nearest_neighbors=10, fields="embedding")

    results = search_client.search(
        search_text=question,
        semantic_configuration_name=azure_ai_search_semantic_configuration_name,
        query_type=QueryType.SEMANTIC,
        top=10,
        select=["content","source","category"],
        vector_queries=[vector_query],
    ) 
    results_list = [
        {
        "content": doc["content"],
        "source": doc["source"],
        "category": doc["category"],
        "search_score": doc["@search.score"]
        }
    for doc in results]

    # Sort the results by search_score in descending order
    sorted_results = sorted(results_list, key=lambda x: x['search_score'], reverse=True)

    # loop through the sorted results and check if there is any source value that starts with "FAQ", if so return the top 1 result, else return the top 5 results
    for result in sorted_results:
        if result["source"].startswith("FAQ"):
            return result
    return sorted_results[:5]

search_ai_index("ACS880 总线控制如何获取EDS & GSD 文件？")

{'content': '问题：ACS880 总线控制如何获取EDS & GSD 文件？ 答案：通常情况下，GSD文件可以通过ABB官方网站直接下载，EDS文件可以使用Drivecomposerpro1.5版本，用户自己生成。',
 'source': 'FAQ_Q17',
 'category': 'ACS880',
 'search_score': 0.03333333507180214}